In [ ]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
import numpy as np
import os
import scipy.io
from skimage import color
from skimage import io
from PIL import Image as im

In [ ]:
plt.rcParams['figure.figsize'] = [8, 8]
plt.rcParams.update({'font.size': 18})

mat_contents = scipy.io.loadmat(os.path.join('allFaces.mat'))
faces = mat_contents['faces']
m = int(mat_contents['m'])
n = int(mat_contents['n'])
nfaces = np.ndarray.flatten(mat_contents['nfaces'])

# We use the first 36 people for training data
trainingFaces = faces[:,:np.sum(nfaces[:36])]
avgFace = np.mean(trainingFaces,axis=1) # size n*m by 1

# Compute eigenfaces on mean-subtracted training data
X = trainingFaces - np.tile(avgFace,(trainingFaces.shape[1],1)).T
U, S, VT = np.linalg.svd(X,full_matrices=0)

fig1 = plt.figure()
ax1 = fig1.add_subplot(121)
img_avg = ax1.imshow(np.reshape(avgFace,(m,n)).T)
img_avg.set_cmap('gray')
plt.axis('off')

ax2 = fig1.add_subplot(122)
img_u1 = ax2.imshow(np.reshape(U[:,0],(m,n)).T)
img_u1.set_cmap('gray')
plt.axis('off')

plt.show()

In [ ]:
img = plt.imshow(np.reshape(U[:,0],(m,n)).T)
img.set_cmap('gray')
plt.axis('off')
plt.show()

In [ ]:
sizeEFs = (n*8, m*8)
EigenFaces = np.zeros(sizeEFs)
count = 1

imagelist = []
index = []

for i in range(1,9):
    for j in range(1,9):
        EigenFaces[(i-1)*n : i*n , (j-1)*m : j*m] = np.reshape(U[: , count], (n,m))
        count = count + 1

for i in range(1,9):
    for j in range(1,9):
        index.append('img' + str(i+j-1))
        imagelist.append(np.reshape(EigenFaces[(i-1)*192:i*192,(j-1)*168:j*168],(m,n)).T)

dictionary = dict(zip(index, imagelist))

for q in range(1,64):
    plt.imshow(imagelist[q])
    plt.set_cmap('gray')

In [ ]:
for z in range(1,64):
    fig = plt.figure(figsize=(192, 168))
    ax = fig.add_subplot(8, 8, z)
    imgplot = plt.imshow(imagelist[z-1])
    plt.axis('off')

In [ ]:
test_tup1 = EigenFaces.shape
test_tup2 = (8,8)
res = tuple(ele1 // ele2 for ele1, ele2 in zip(test_tup1, test_tup2))
res
test_tup1

In [ ]:
## Now show eigenface reconstruction of image that was omitted from test set

img0 = io.imread('dog.jpg')
img1 = color.rgb2gray(img0).astype(float)
img2 = img1.T
img3 = img2.flatten()
testFace = img3

plt.imshow(np.reshape(testFace,(m,n)).T)
plt.set_cmap('gray')
plt.title('Original Image')
plt.axis('off')
plt.show()

testFaceMS = testFace - avgFace
r_list = [25, 16000]

for r in r_list:
    reconFace = avgFace + U[:,:r]  @ U[:,:r].T @ testFaceMS
    img = plt.imshow(np.reshape(reconFace,(m,n)).T)
    img.set_cmap('gray')
    plt.title('r = ' + str(r))
    plt.axis('off')
    plt.show()

In [ ]:
## Project person 2 and 7 onto PC5 and PC6

P1num = 2 # Person number 2
P2num = 7 # Person number 7

P1 = faces[:,np.sum(nfaces[:(P1num-1)]):np.sum(nfaces[:P1num])]
P2 = faces[:,np.sum(nfaces[:(P2num-1)]):np.sum(nfaces[:P2num])]

P1 = P1 - np.tile(avgFace,(P1.shape[1],1)).T
P2 = P2 - np.tile(avgFace,(P2.shape[1],1)).T

PCAmodes = [5, 6] # Project onto PCA modes 5 and 6
PCACoordsP1 = U[:,PCAmodes-np.ones_like(PCAmodes)].T @ P1
PCACoordsP2 = U[:,PCAmodes-np.ones_like(PCAmodes)].T @ P2

plt.plot(PCACoordsP1[0,:],PCACoordsP1[1,:],'d',Color='k',label='Person 2')
plt.plot(PCACoordsP2[0,:],PCACoordsP2[1,:],'^',Color='r',label='Person 7')

plt.legend()
plt.show()